In [3]:
from theano import *
from lasagne.layers import EmbeddingLayer, InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np

In [4]:
l_in = lasagne.layers.InputLayer((10,50))

In [5]:
l_output = lasagne.layers.DenseLayer(l_in, num_units=2, name='output')

/home/matthew/.virtualenvs/nlp-convnet/lib/python2.7/site-packages/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "


In [6]:
out = lasagne.layers.get_output(l_output)

/home/matthew/.virtualenvs/nlp-convnet/lib/python2.7/site-packages/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "


In [7]:
print out

Elemwise{mul,no_inplace}.0


In [8]:
func = theano.function([l_in.input_var], out)

In [9]:
target_v = (np.random.randint(0,2,10)).astype('int32')
target_t = np.random.randn(10, 50)

In [10]:
target_v.shape, target_t.shape

((10,), (10, 50))

In [11]:
func(target_t)

array([[ 0.        ,  0.        ],
       [ 0.        ,  1.06923009],
       [ 0.        ,  0.23545635],
       [ 0.        ,  0.34991309],
       [ 0.71307727,  1.69669599],
       [ 0.        ,  0.        ],
       [ 0.83455082,  0.99672896],
       [ 2.02694026,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.93246807]])

In [12]:
all_params = lasagne.layers.get_all_params(l_output)

In [13]:
theano.function([l_in.input_var], (out, T.argmax(out, axis=1)))(target_t)[1]

array([0, 1, 1, 1, 1, 0, 1, 0, 0, 1])

In [14]:
y_value = T.ivector('y')
out_values = T.argmax(out, axis=1)
#loss = (y_value - out_values) ** 2
#loss_old = (y_value - ) ** 2
loss = theano.tensor.nnet.binary_crossentropy(out_values + 1, y_value + 1)

There was some issue with the shape of the ouput vector being [10,1] and the shape of the Y values being [10]

this hack above is computing the loss by transforming the output vector into a single dimentional item, and then we compute the difference between these two terms and square it

maybe simply taking the transpose of the y_value matrix can make these two values compatiable

In [15]:
update = lasagne.updates.nesterov_momentum(loss.mean(), all_params, 0.1)

In [16]:
train = theano.function([l_in.input_var, y_value], (out_values, y_value, loss, loss.mean(), update.values()[0]), updates=update)

In [17]:
update

OrderedDict([(output.W, Elemwise{add,no_inplace}.0), (output.b, Elemwise{add,no_inplace}.0), (<TensorType(float64, matrix)>, Elemwise{sub,no_inplace}.0), (<TensorType(float64, vector)>, Elemwise{sub,no_inplace}.0)])

In [18]:
for i in xrange(1000):
    _,_,_,loss_v,_= train(target_t, target_v)
print loss_v

nan


In [19]:
train(target_t, target_v)

[array([0, 1, 1, 1, 1, 0, 1, 0, 0, 1]),
 array([0, 1, 1, 1, 1, 1, 1, 0, 1, 0], dtype=int32),
 array([ nan,  nan,  nan,  nan,  nan, -inf,  nan,  nan, -inf,  nan]),
 array(nan),
 array([[ 0.01455541, -0.06080716],
        [-0.04779141, -0.01968734],
        [-0.27181563, -0.19865111],
        [-0.09148882, -0.17334739],
        [ 0.32892308,  0.21018575],
        [-0.2973821 , -0.32153287],
        [ 0.25786338,  0.04905461],
        [ 0.08217621,  0.29279125],
        [-0.11245267, -0.01823565],
        [-0.05138954,  0.05137051],
        [-0.00736179, -0.03140707],
        [-0.12530164,  0.11736027],
        [-0.04462106,  0.00109277],
        [-0.12288981,  0.27872435],
        [-0.26749214,  0.33124063],
        [-0.23587184, -0.15745308],
        [-0.02044665,  0.07170847],
        [ 0.00300819,  0.19918474],
        [ 0.11786037,  0.00308929],
        [ 0.23629698,  0.24807301],
        [-0.03601979, -0.27971277],
        [ 0.06338235,  0.09053476],
        [ 0.20848981,  0.2057562

In [20]:
func(target_t), target_v

(array([[ 0.        ,  0.        ],
        [ 0.        ,  1.06923009],
        [ 0.        ,  0.23545635],
        [ 0.        ,  0.34991309],
        [ 0.71307727,  1.69669599],
        [ 0.        ,  0.        ],
        [ 0.83455082,  0.99672896],
        [ 2.02694026,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.93246807]]),
 array([0, 1, 1, 1, 1, 1, 1, 0, 1, 0], dtype=int32))

In [21]:
theano.config.device

[autoreload of theano.gof.optdb failed: Traceback (most recent call last):
  File "/usr/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name DefaultOrderedDict
]
[autoreload of theano.scan_module.scan failed: Traceback (most recent call last):
  File "/usr/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name ifilter
]
[autoreload of theano.sandbox.rng_mrg failed: Traceback (most recent call last):
  File "/usr/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name multinomial
]
[autoreload of theano.ifelse failed: Traceback (most recent call last):
  File "/usr/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.ol

Couldn't import dot_parser, loading of dot files will not be possible.



[autoreload of theano.gof.link failed: Traceback (most recent call last):
  File "/usr/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name izip
]
[autoreload of theano failed: Traceback (most recent call last):
  File "/usr/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
AttributeError: 'module' object has no attribute 'tests'
]
[autoreload of lasagne.layers.pool failed: Traceback (most recent call last):
  File "/usr/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name downsample
]
[autoreload of theano.sandbox.cuda.rng_curand failed: Traceback (most recent call last):
  File "/usr/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old

AttributeError: 'TheanoConfigParser' object has no attribute 'device'